In [ ]:
# Import necessary libraries
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import os
import matplotlib.pyplot as plt

# Load movie dataset into a DataFrame
df = pd.read_csv("data/movies.csv")

# Parse release_date into datetime objects and extract the release month
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["release_month"] = df["release_date"].dt.month

# Drop any rows missing budget, revenue, or month — these are key inputs for modeling
df = df.dropna(subset=["budget", "revenue", "release_month"])

# Create a binary target column: is_hit = True if revenue exceeds some budget multiple
# We iterate through several thresholds and pick the first one with both classes present
thresholds = [2.0, 1.5, 1.0, 0.8]
success = False

for t in thresholds:
    df["is_hit"] = df["revenue"] > (t * df["budget"])
    class_counts = df["is_hit"].value_counts()

    # Check if the current threshold results in both True and False labels
    if len(class_counts) >= 2:
        print(f"Using threshold: revenue > {t} * budget")
        success = True
        break
    else:
        print(f"Only one class at threshold {t}x: {class_counts.to_dict()}")

# If no usable class split is found, abort the process
if not success:
    raise ValueError("No usable class distribution found at any threshold.")

# Define features and target
X = df[["budget", "release_month"]]
y = df["is_hit"]

# Split the dataset into training and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize feature values to improve model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a logistic regression model on the scaled data
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Save the trained model and scaler for reuse (e.g., in deployment)
os.makedirs("ml", exist_ok=True)
joblib.dump((model, scaler), "ml/hit_predictor.pkl")
print("Trained model saved to: ml/hit_predictor.pkl")

# Predict on the test set and print evaluation metrics
y_pred = model.predict(X_test_scaled)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# Visualize performance with a confusion matrix
ConfusionMatrixDisplay.from_estimator(model, X_test_scaled, y_test)
plt.title("Confusion Matrix")
plt.show()